In [ ]:
import ctypes
import numpy as np
import time
lib = ctypes.cdll.LoadLibrary("./lib_matrix.py")


OSError: dlopen(./libmatrix.so, 0x0006): tried: './libmatrix.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OS./libmatrix.so' (no such file), '/opt/anaconda3/lib/python3.12/lib-dynload/../.././libmatrix.so' (no such file), '/opt/anaconda3/bin/../lib/./libmatrix.so' (no such file), '/usr/lib/./libmatrix.so' (no such file, not in dyld cache), './libmatrix.so' (no such file), '/Users/ryansilva/Downloads/560-Lab-3/scripts/libmatrix.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/ryansilva/Downloads/560-Lab-3/scripts/libmatrix.so' (no such file), '/Users/ryansilva/Downloads/560-Lab-3/scripts/libmatrix.so' (no such file)

In [ ]:
lib.imageConvNaive.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags="C_CONTIGUOUS"),
    ctypes.c_int,
    ctypes.c_int,
    ctypes.c_int
]


In [ ]:
img_path = "../data/input_512.pgm"
with open(img_path, "rb") as f:
    assert f.readline().strip() == b"P5"
    # skip comments
    line = f.readline()
    while line.startswith(b"#"):
        line = f.readline()
    w, h = map(int, line.split())
    maxval = int(f.readline())
    img = np.frombuffer(f.read(), dtype=np.uint8)
    img = img.reshape((h, w)).astype(np.float32)


# --------------------------------------------------
# Load real input image
# --------------------------------------------------
   # or input_256.pgm / input_1024.pgm
input_img = img

H, W = input_img.shape
filter_dim = 1  # radius => 3x3 filter

kernel = np.random.rand(
    2 * filter_dim + 1,
    2 * filter_dim + 1
).astype(np.float32)

output_img = np.zeros((H, W), dtype=np.float32)


In [ ]:
H, W = 512, 512
filter_dim = 1  # radius => 3x3 filter

# Input image (random for now)
input_img = np.random.rand(H, W).astype(np.float32)

# Filter
kernel = np.random.rand(
    2 * filter_dim + 1,
    2 * filter_dim + 1
).astype(np.float32)

# Output buffer
output_img = np.zeros((H, W), dtype=np.float32)

# -----------------------------
# Run CUDA convolution
# -----------------------------
start = time.time()
lib.imageConvNaive(
    input_img.ravel(),
    kernel.ravel(),
    output_img.ravel(),
    H,
    W,
    filter_dim
)
end = time.time()

print(f"Runtime: {end - start:.4f} seconds")

# -----------------------------
# Save output as PGM
# -----------------------------
def save_pgm(path, img):
    img = np.clip(img, 0, 255).astype(np.uint8)
    h, w = img.shape
    with open(path, "wb") as f:
        f.write(b"P5\n")
        f.write(f"{w} {h}\n255\n".encode())
        f.write(img.tobytes())

save_pgm("../results/output_512.pgm", outpu
